In [4]:
import pandas as pd
import plotly

import asana
import wa_access as wac

In [5]:
def extract_custom_field_vals(cf_list):
    vals_dict = dict()
    for c in cf_list: 
        c_type = c['type']
        if c_type == 'enum':
            c_val = c['enum_value']
            vals_dict[c['name']] = c_val['name'] if c_val is not None else 'none'

        elif c_type == 'text':
            c_val = c['text_value']
            vals_dict[c['name']] = c_val if c_val is not None else 'none'

        elif c_type == 'number':
            c_val = c['number_value']
            vals_dict[c['name']] = c_val if c_val is not None else 'none'
        
    return vals_dict

In [6]:
def get_global_portfolio_data():
    dh_wart_key = wac.get_asana_pat()
    wartsila_workspace = '8836488480224'
    global_portfolio_gid = '1201827339230998'

    #establish asana client
    client = asana.Client.access_token(dh_wart_key)

    #task_fields = ['name','gid','assignee.name','start_on','due_on', 'custom_fields', 'projects.name']
    #wrong - global_tasks = client.tasks.get_tasks_for_project(global_project_gid) #, opt_fields=task_fields) # 'custom_fields'])
    
    portfolio_fields = ['custom_fields']
    global_portfolio_cfs = client.portfolios.get_items_for_portfolio(global_portfolio_gid, opt_fields=portfolio_fields)
    
    global_portfolio_names = client.portfolios.get_items_for_portfolio(global_portfolio_gid)
    
    # unpack Portfolio Project Names API call
    global_portfolio_names_list = list()
    for p in global_portfolio_names:
        global_portfolio_names_list.append(p)
    df_global_pjs = pd.DataFrame(global_portfolio_names_list)
    
    # unpack Project Custom Fields API Call
    gpcl = list()
    for pc in global_portfolio_cfs:
        gpcl.append(pc)
    
    # extract custom field values
    pj_ext_cf_list = list()
    for g in gpcl:
        t_cfs = g['custom_fields']
        pj_dict = extract_custom_field_vals(t_cfs)
        pj_dict['gid'] = g['gid']
        pj_ext_cf_list.append(pj_dict)
    df_global_cfs = pd.DataFrame(pj_ext_cf_list)
    
    # merge Project Names df with Custom Fields df
    df_global = pd.merge(df_global_pjs, df_global_cfs, how='left', on='gid')
    
    return df_global

In [7]:
get_global_portfolio_data()

gid                                               name  \
0   1202504529581315                          P/22454 Clearway Rosamond   
1   1202230203978012                        P/22202 Kavalan - Supernova   
2   1202894707915267                P/776824 - Leopard Cat - duplicate?   
3   1202894707915271                                   P/790697 - Yilan   
4   1202509436142461  P/22208 CUC Cayman Isl. - Prospect Substation ...   
..               ...                                                ...   
57  1202710346311305                    OP/754771 - Pivot Power, Sundon   
58  1202710346311309                   OP/798722 - Pathfinder Kilmanock   
59  1202710346311313                 OP/753544 - Eurus Saltbrush Plains   
60  1202710346311317                             OP/791070 - TECO_Dover   
61  1202901532753744                          P/22454 Clearway Rosamond   

   resource_type    [ESO] Stage Contract COD [ESO]  \
0        project           none               none   
1        project  G2 - Delivery               none   
2        project  G2 - Delivery               none   
3        project  G2 - Delivery               none   
4        project  G2 - Delivery               none   
..           ...            ...                ...   
57       project              A               none   
58       project              A               none   
59       project              A               none   
60       project              A               none   
61       project           none               none   

         City/State/Country [ESO] Priority Project Type [ESO] Region [ESO]  \
0                            none     none               none         none   
1                            none     none               none         none   
2                  Miaoli, Taiwan     none               none         none   
3   Wujie Township, Yilan, Taiwan     none               none         none   
4                  Cayman Islands     none               none         AMER   
..                            ...      ...                ...          ...   
57                 United Kingdom     none                EEQ       Global   
58                 United Kingdom     none                EEQ       Global   
59                             CA     none                EEQ       Global   
60                        Florida     none                EEQ       Global   
61                           none     none               none         none   

   Power (MW) [ESO] Capacity (MWh) [ESO] Batt Enclosure (qty)  \
0              none                 none                 none   
1              none                 none                 none   
2                 5                    5                 none   
3               103                  188                 none   
4                10                   10                 none   
..              ...                  ...                  ...   
57               50                  100                 none   
58              200                  400                 none   
59               10                   40                 none   
60               15                   30                 none   
61             none                 none                 none   

   Inverter Qty [ESO] ACC/DCC Qty Product Role [ESO] Delivery Roles [ESO]  \
0                none        none    none        NaN                  NaN   
1                none        none    none        NaN                  NaN   
2                none        none    none       none                  NaN   
3                none        none    none       none                  NaN   
4                none        none    none        NaN                  NaN   
..                ...         ...     ...        ...                  ...   
57               none        none    none       none                  NaN   
58               none        none    none       none                  NaN   
59               none        none    none       none    